In [1]:
import pandas as pd

pd.set_option('display.max_columns', 500)

In [ ]:
#Get FantasyPros rankings and load to df
import pandas as pd
import glob
import os 
import numpy as np

# Use glob to create a list of file paths for all .csv files in the specified folder
path = 'C:/Users/onine/Downloads/*.csv'
csv_files = glob.glob(path)
df2 = pd.DataFrame()

for file in csv_files:
    if 'nba-player-props-rotowire' in file:
        
        # Read in the file
        df_temp = pd.read_csv(file, skiprows=1)
        stat = df_temp.columns[3]
        
        bet_const = 115
        bet_div = 30
        
        if stat in ['Rebounds','Assists']:
            bet_div = 45
        
        if stat in ['Turnovers']:
            bet_div = 60
        
        if stat in ['Steals','Blocks']:
            bet_div = 90
        
        print(stat)
        
        columns_to_check = ['Over', 'Under']
        df_temp['Greater'] = df_temp[columns_to_check].abs().idxmax(axis=1)
        df_temp['Greater_Value'] = df_temp[columns_to_check].abs().max(axis=1)
        df_temp['Change'] = np.where(df_temp['Greater']=='Over',1,-1)
        df_temp[stat] = df_temp[stat] + (df_temp['Change']*(df_temp['Greater_Value'].abs()-bet_const)/bet_div)
        
        columns_to_check = ['Over.1', 'Under.1']
        df_temp['Greater.1'] = df_temp[columns_to_check].abs().idxmax(axis=1)
        df_temp['Greater_Value.1'] = df_temp[columns_to_check].abs().max(axis=1)
        df_temp['Change.1'] = np.where(df_temp['Greater.1']=='Over.1',1,-1)
        df_temp[stat+'.1'] = df_temp[stat+'.1'] + (df_temp['Change.1']*(df_temp['Greater_Value.1'].abs()-bet_const)/bet_div)
        
        columns_to_check = ['Over.2', 'Under.2']
        df_temp['Greater.2'] = df_temp[columns_to_check].abs().idxmax(axis=1)
        df_temp['Greater_Value.2'] = df_temp[columns_to_check].abs().max(axis=1)
        df_temp['Change.2'] = np.where(df_temp['Greater.2']=='Over.2',1,-1)
        df_temp[stat+'.2'] = df_temp[stat+'.2'] + (df_temp['Change.2']*(df_temp['Greater_Value.2'].abs()-bet_const)/bet_div)
        
        columns_to_check = ['Over.3', 'Under.3']
        df_temp['Greater.3'] = df_temp[columns_to_check].abs().idxmax(axis=1)
        df_temp['Greater_Value.3'] = df_temp[columns_to_check].abs().max(axis=1)
        df_temp['Change.3'] = np.where(df_temp['Greater.3']=='Over.3',1,-1)
        df_temp[stat+'.3'] = df_temp[stat+'.3'] + (df_temp['Change.3']*(df_temp['Greater_Value.3'].abs()-bet_const)/bet_div)

        #df_temp.columns = ['Player','Team','Opp',str(stat),'Over1','Under1','Remove1','Over2','Under2','Remove2','Over2','Under2','Remove','Over3','Under3']
        
        # Concatenate the dataframes
        df2 = pd.concat([df2, df_temp])
        os.remove(file)

df2.head(4)

In [ ]:
df3 = df2[['Player','Team','Opp','Rebounds','Rebounds.1','Rebounds.2','Rebounds.3','Points','Points.1','Points.2','Points.3','Assists','Assists.1','Assists.2','Assists.3','Turnovers','Turnovers.1','Turnovers.2','Turnovers.3','Steals','Steals.1','Steals.2','Steals.3','Blocks','Blocks.1','Blocks.2','Blocks.3']]
df3[['Rebounds','Rebounds.1','Rebounds.2','Rebounds.3','Points','Points.1','Points.2','Points.3','Assists','Assists.1','Assists.2','Assists.3','Turnovers','Turnovers.1','Turnovers.2','Turnovers.3','Steals','Steals.1','Steals.2','Steals.3','Blocks','Blocks.1','Blocks.2','Blocks.3']] = df3[['Rebounds','Rebounds.1','Rebounds.2','Rebounds.3','Points','Points.1','Points.2','Points.3','Assists','Assists.1','Assists.2','Assists.3','Turnovers','Turnovers.1','Turnovers.2','Turnovers.3','Steals','Steals.1','Steals.2','Steals.3','Blocks','Blocks.1','Blocks.2','Blocks.3']].clip(lower=0)

df3 = df3.groupby(['Player','Team','Opp']).min().reset_index()

df3['Rebounds_min'] = df3[['Rebounds','Rebounds.1','Rebounds.2','Rebounds.3']].mean(axis=1)
df3['Points_min'] = df3[['Points','Points.1','Points.2','Points.3']].mean(axis=1)
df3['Assists_min'] = df3[['Assists','Assists.1','Assists.2','Assists.3']].mean(axis=1)
df3['Blocks_min'] = df3[['Blocks','Blocks.1','Blocks.2','Blocks.3']].mean(axis=1)
df3['Steals_min'] = df3[['Steals','Steals.1','Steals.2','Steals.3']].mean(axis=1)
df3['Turnovers_max'] = df3[['Turnovers','Turnovers.1','Turnovers.2','Turnovers.3']].mean(axis=1)

df3['Rebounds_pts'] = df3['Rebounds_min']*1.2
df3['Points_pts'] = df3['Points_min']
df3['Assists_pts'] = df3['Assists_min']*1.5
df3['Blocks_pts'] = df3['Blocks_min']*3
df3['Steals_pts'] = df3['Steals_min']*3
df3['Turnovers_pts'] = df3['Turnovers_max']

df3['Total'] = df3['Rebounds_pts'].fillna(0)+df3['Points_pts'].fillna(0)+df3['Assists_pts'].fillna(0)+df3['Blocks_pts'].fillna(0)+df3['Steals_pts'].fillna(0)-df3['Turnovers_pts'].fillna(0)
df3['Key'] = df3['Player'] + '-' + df3['Team']
df3.head()

In [ ]:
import requests
from bs4 import BeautifulSoup

html = requests.get("https://www.fantasypros.com/daily-fantasy/nba/fanduel-salary-changes.php").content
soup = BeautifulSoup(html, "html.parser")
table = soup.find("table")
prices_df = pd.read_html(str(table))[0]

prices_df['Name'] = prices_df['Player'].str.split('(').str[0].str[0:-1]
prices_df['Name'] = prices_df['Name'].str.replace(' Jr.','')
prices_df['Name'] = prices_df['Name'].str.replace(' Sr.','')
prices_df['Name'] = prices_df['Name'].str.replace(' III','')

prices_df['Team'] = prices_df['Player'].apply(lambda st: st[st.find("(")+1:st.find(" - ")])
prices_df['Team'] = prices_df['Team'].str.replace('UTH', 'UTA')
prices_df['Team'] = prices_df['Team'].str.replace('NOR', 'NOP')

prices_df['Pos'] = prices_df['Player'].apply(lambda st: st[st.find(" - ")+3:st.find(")")])
prices_df['Key'] = prices_df['Name'] + '-' + prices_df['Team']

prices_df['Price'] = prices_df['Today'].replace("[$,]", "", regex=True).astype(int)

prices_df.to_csv('Fanduel_Player_Prices_Clean.csv', index=False)
prices_df.head()

In [ ]:
df4 = pd.merge(df3, prices_df, how='left', on='Key')
df4['Value'] = df4['Total']/df4['Price']*1000 
df4.sort_values(by='Value', ascending=False)

In [ ]:
df4.to_csv('test.csv')

In [ ]:
from pulp import *

In [13]:
df4 = pd.read_csv('test.csv')
players = df4[['Key','Price','Pos','Total']]
players[['Pos 1', 'Pos 2']] = players['Pos'].str.split("/", expand=True)
players = players.drop(columns=['Pos'])
players['player'] = players.index
players = players[players['Price'] < 13000]
#players.rename(columns={"Key": "player"}, inplace=True)
players.head()

C:\Users\onine\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Key,Price,Total,Pos 1,Pos 2,player
0,Aaron Gordon-DEN,6400.0,35.456667,PF,C,0
1,Alex Caruso-CHI,4400.0,21.598333,PG,SG,1
2,Alperen Sengun-HOU,6400.0,36.133333,C,None,2
3,Andrew Wiggins-GSW,7000.0,30.386667,SF,PF,3
4,Ayo Dosunmu-CHI,4300.0,21.088889,PG,SG,4


In [14]:

# create the optimization model
model = LpProblem("Team Optimization", LpMaximize)

# create decision variables for each player
player_vars = {player: LpVariable(f"{player}_selected", 0, 1, LpBinary) for player in players['player']}

# create decision variables for each position
pg_var = LpVariable("PG_selected", 2, 2, LpInteger)
sg_var = LpVariable("SG_selected", 2, 2, LpInteger)
sf_var = LpVariable("SF_selected", 2, 2, LpInteger)
pf_var = LpVariable("PF_selected", 2, 2, LpInteger)
c_var = LpVariable("C_selected", 1, 1, LpInteger)

# add the salary cap constraint
model += lpSum([players.loc[i, 'Price'] * player_vars[i] for i in players.index]) <= 60000

# add the position constraint
model += lpSum([player_vars[i] for i in players.index if players.loc[i, 'Pos 1'] == 'PG']) == pg_var
model += lpSum([player_vars[i] for i in players.index if players.loc[i, 'Pos 1'] == 'SG']) == sg_var
model += lpSum([player_vars[i] for i in players.index if players.loc[i, 'Pos 1'] == 'SF']) == sf_var
model += lpSum([player_vars[i] for i in players.index if players.loc[i, 'Pos 1'] == 'PF']) == pf_var
model += lpSum([player_vars[i] for i in players.index if players.loc[i, 'Pos 1'] == 'C']) == c_var

# add the constraint that a player can only play one of the two positions
#for i in players.index:
#    model += player_vars[i] <= players.loc[i, 'Pos 1']
#    model += player_vars[i] <= players.loc[i, 'Pos 2']

# set the objective function
model += lpSum([players.loc[i, 'Total'] * player_vars[i] for i in players.index]) + lpSum([pg_var, sg_var, sf_var, pf_var, c_var]), "Total Points"

#solve the model
model.solve()

#print the results
print("Status:", LpStatus[model.status])
for player, var in player_vars.items():
    if var.varValue == 1:
        print("Player selected:", player)

# print the results
print("Number of PGs selected:", pg_var.varValue)
print("Number of SGs selected:", sg_var.varValue)
print("Number of SFs selected:", sf_var.varValue)
print("Number of PFs selected:", pf_var.varValue)
print("Number of Cs selected:", c_var.varValue)

# print the total salary and points of the selected team
print("Total Salary:", value(lpSum([players.loc[i, 'Price'] * player_vars[i] for i in players.index])))
print("Total Points:", value(lpSum([players.loc[i, 'Total'] * player_vars[i] for i in players.index])))

Status: Optimal
Player selected: 0
Player selected: 2
Player selected: 5
Player selected: 10
Player selected: 11
Player selected: 23
Player selected: 27
Player selected: 62
Player selected: 77
Number of PGs selected: 2.0
Number of SGs selected: 2.0
Number of SFs selected: 2.0
Number of PFs selected: 2.0
Number of Cs selected: 1.0
Total Salary: 60000.0
Total Points: 307.89166666666665


C:\Users\onine\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [15]:
player_list = []

# Print the selected players
for player in players.player:
    if player_vars[player].varValue == 1.0:
        player_list.append(player)
        #print(player)

players.loc[player_list]

,Key,Price,Total,Pos 1,Pos 2,player
0,Aaron Gordon-DEN,6400.0,35.456667,PF,C,0
2,Alperen Sengun-HOU,6400.0,36.133333,C,None,2
5,Bogdan Bogdanovic-ATL,6000.0,30.051667,SF,SG,5
10,D'Angelo Russell-MIN,6900.0,32.876667,PG,SG,10
11,De'Aaron Fox-SAC,8500.0,42.196667,PG,None,11
23,Jabari Smith-HOU,5200.0,24.456667,PF,None,23
27,Jalen Green-HOU,7000.0,36.180000,SG,PG,27
62,Patrick Williams-CHI,5000.0,25.523333,SF,PF,62
77,Zach LaVine-CHI,8600.0,45.016667,SG,SF,77


In [ ]:
from pulp import LpVariable, LpProblem, LpMaximize, LpInteger, lpSum
import pandas as pd

# Create the problem
prob = LpProblem("Team Optimization Problem", LpMaximize)

# Create a binary variable for each player indicating whether they are on the team
player_vars = {player: LpVariable(f"x_{player}", 0, 1, LpInteger) for player in players.player}

# Add a constraint for the salary cap
prob += lpSum([players.at[player, "Price"] * player_vars[player] for player in players.player]) <= 60000, "Salary Cap"

# Add constraints for the number of players at each position
positions = ["PG", "SG", "SF", "PF", "C"]

for position in positions:
    prob += lpSum([player_vars[player] for player in players.player if position in (players.at[player, "Pos 1"], players.at[player, "Pos 2"])]) == 1, f"{position} Count"

# Objective function to maximize points
prob += lpSum([players.at[player, "Total"] * player_vars[player] for player in players.player]), "Total Points"

# Solve the problem
prob.solve()


In [ ]:
from pulp import *

# Create a list of players with their positions, salary, and points
players = [("Player1", "PG", "SG", 5000, 20),
           ("Player2", "SG", "SF", 6000, 25),
           ("Player3", "SF", "PF", 7000, 30),
           ("Player4", "PF", "C", 8000, 35),
           ("Player5", "C", "PF", 9000, 40),
           # Add the rest of the players here
          ]

# Create the problem object
prob = LpProblem("Team Optimization Problem", LpMaximize)

# Create decision variables for each player
player_vars = LpVariable.dicts("Player", [p[0] for p in players], cat="Binary")

# Add the objective function to maximize points
prob += lpSum([p[4]*player_vars[p[0]] for p in players])

# Add constraint for the salary cap
prob += lpSum([p[3]*player_vars[p[0]] for p in players]) <= 60000

# Add constraints for each position
for pos in ["PG", "SG", "SF", "PF", "C"]:
    prob += lpSum([player_vars[p[0]] for p in players if pos in [p[1], p[2]]]) == 1

# Solve the problem
prob.solve()

# Print the solution
print("Team Optimization Problem")
print("Status:", LpStatus[prob.status])
print("Maximum Points:", value(prob.objective))
print("Selected Players:")
for p in players:
    if player_vars[p[0]].varValue:
        print(p[0])
